### Workbook for estimating population counts in villages surrounding Rwanda bridges
Week of April 16, 2025
Author: Adele Birkenes

Step 1: Import packages & read in data on village boundaries, population (WorldPop raster), and bridges

In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.plot import show
import os

In [12]:
path = "../../synced-data/population-exploration/"

# Village boundaries (geodataframe)
village_boundaries_filename = os.path.join(path,"Rwanda Village Boundaries/Village.shp")
Rwanda_village_boundaries = gpd.read_file(os.path.join(path, village_boundaries_filename))

# Bridges (dataframe)
bridges_filename = os.path.join(path, "rct-all-bridges.csv")
Rwanda_bridges = pd.read_csv(os.path.join(path, bridges_filename))

# Population
rwa_pop_2020_filename = os.path.join(path, "rwa_ppp_2020.tif")

Step 2: Convert bridges dataframe to geodataframe & match projection to that of village boundaries

In [23]:
def map_bridges(bridges, village_boundaries):

    # Check CRS of village boundaries gdf
    print(f'The CRS of the village boundaries gdf is: {village_boundaries.crs}')

    # Create lat/lon variables
    lon = bridges['GPS (Longitude)']
    lat = bridges['GPS (Latitude)']

    # Create gdf of bridges data by converting lat/lon values to list of Shapely Point objects
    bridge_points = gpd.GeoDataFrame(bridges, geometry=gpd.points_from_xy(x=lon, y=lat), crs='EPSG:4326')

    # Set CRS of bridges gdf to CRS of village boundaries gdf
    bridge_points.to_crs(village_boundaries.crs, inplace=True)

    # Check that reprojection was successful
    print(f'The CRS of the bridges gdf is: {bridge_points.crs}')

    return bridge_points

bridge_points = map_bridges(bridges = Rwanda_bridges, village_boundaries = Rwanda_village_boundaries)

The CRS of the village boundaries gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
The CRS of the bridges gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_

Step 3: Create buffers around bridges according to user's distance input

In [25]:
def create_bridge_buffers(bridges, buffer_distance):

    # Create buffers around bridges according to user's distance input
    buf = bridges.geometry.buffer(distance=buffer_distance)

    # Add bridges point attribute information to buffers
    bridge_buffers = bridges_points.copy()
    bridge_buffers['geometry'] = buf

    return bridge_buffers

bridge_buffers = create_bridge_buffers(bridges = bridge_points, buffer_distance = 2000)
bridge_buffers.head()

,Opportunity Name,Randomized Order,Research Initiative,Bridge Name,GPS (Latitude),GPS (Longitude),Level 1 Government,Level 3 Government,Level 4 Government,Stage,Sub-Stage,Individuals Directly Served,Bridge Type,Span (m),Ground Breaking Date,Close Date,CaseSafeID,Project Code,Close Month,B2P Fiscal Year,Level 2 Government,geometry
0,Rwanda - Cyabami (Kwa Anamariya) - 1013233,1.0,Rwanda Full Scale Research - Randomized and Ba...,Cyabami (Kwa Anamariya),-1.608340,29.891100,Northern Province,Nemba,Kadehero,Complete,In Service,2554.0,Suspended Bridge,56.0,5/24/2022,9/8/2022,006f100000d7608AAA,1013233,9/1/2022,2023,Burera,"POLYGON ((489883.256 4822175.961, 489873.625 4..."
1,Rwanda - Nyabizi ya 1 - 1013496,2.0,Rwanda Full Scale Research - Randomized and Ba...,Nyabizi ya 1,-1.330010,29.825550,Northern Province,Kinyababa,Rutovu,Complete,In Service,1854.0,Suspended Bridge,60.0,6/10/2022,10/27/2022,006f100000d7Q9rAAE,1013496,10/1/2022,2023,Burera,"POLYGON ((482587.423 4852948.863, 482577.792 4..."
2,Rwanda - Banga - 1012550,18.0,Rwanda Full Scale Research - Randomized and Ba...,Banga,-1.689722,29.808880,Northern Province,Gakenke-Minazi,Ndora-mutara,Identified,Requested,600.0,NaN,NaN,NaN,12/31/2025,006f100000dhpEmAAI,1012550,12/1/2025,2025,Gakenke,"POLYGON ((480735.895 4813177.286, 480726.265 4..."
3,Rwanda - Buranga - 1007356,8.0,Rwanda Full Scale Research - Randomized and Ba...,Buranga,-1.639722,29.784722,Northern Province,Nemba,NaN,Rejected,Technical,2958.0,NaN,NaN,NaN,12/31/2018,006f100000a86DIAAY,1007356,12/1/2018,2018,Gakenke,"POLYGON ((478047.449 4818705.274, 478037.818 4..."
4,Rwanda - Gitwa - 1007340,7.0,Rwanda Full Scale Research - Randomized and Ba...,Gitwa,-1.738888,29.755277,Northern Province,Mataba-minazi,Gitwa/gitwa-nyundo/mwanza,Complete,In Service,2860.0,Suspended Bridge,52.0,5/1/2021,6/30/2021,006f100000a86D2AAI,1007340,6/1/2021,2021,Gakenke,"POLYGON ((474772.661 4807740.589, 474763.031 4..."


Step 4: Perform a spatial join to identify villages that overlap with the bridge buffers and return a df listing each village and its associated bridge IDs

In [28]:
def associate_villages_with_bridges(village_gdf, bridge_buffers_gdf, village_id_col, bridge_id_col):

    # Perform spatial join
    joined = gpd.sjoin(village_gdf, bridge_buffers_gdf, how="inner", predicate="intersects")

    # Group bridge IDs by village ID
    grouped = joined.groupby(village_id_col)[bridge_id_col].apply(list).reset_index()

    # Drop duplicates so each village ID appears only once
    village_attrs = village_gdf.drop_duplicates(subset=village_id_col)

    # Merge grouped bridge lists back into village data
    result = village_attrs.merge(grouped, on=village_id_col, how='left')

    return result

village_bridge_df = associate_villages_with_bridges(village_gdf = Rwanda_village_boundaries, 
                                                    bridge_buffers_gdf = bridge_buffers, 
                                                    village_id_col='Village_ID', 
                                                    bridge_id_col='CaseSafeID')

village_bridge_df.head()

,District,Village_ID,Cell_ID,Sector_ID,Distr_ID,Prov_ID,Name,Cell,Sector,Province,geometry,CaseSafeID
0,Nyarugenge,11010102,110101,1101,11,1,Gihanga,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,"POLYGON Z ((505804.107 4784697.395 0, 505808.8...",NaN
1,Nyarugenge,11010103,110101,1101,11,1,Iterambere,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,"POLYGON Z ((505725.35 4785070.964 0, 505750.76...",NaN
2,Nyarugenge,11010104,110101,1101,11,1,Izuba,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,"POLYGON Z ((505689.339 4785047.225 0, 505712.2...",NaN
3,Nyarugenge,11010105,110101,1101,11,1,Nyaburanga,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,"POLYGON Z ((506082.394 4784751.676 0, 506066.2...",NaN
4,Nyarugenge,11010106,110101,1101,11,1,Nyenyeri,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,"POLYGON Z ((505779.445 4784753.818 0, 505787.7...",NaN
